# A Reverse mode way to compute high order derivatives with OTI numbers
```
Mauricio Aristizabal, PhD
06/22/2023
```

This document implements an initial version of the Reverse mode approach with OTI numbers.

First, import the librarires.

In [1]:
import pyoti.sparse as oti
import pyoti.core as coti
import numpy as np
import math

The test function is the following

$$
f(x_1,\ldots, x_m)=\cos\left(\sin\left(\log\left(\sum_{i=1}^{m} x_i\right)\right)\right)
$$

for a total number of variables $m$.

The following three functions implement the concept using numpy, math and oti in forward and reverse mode.

In [2]:
# *****************************************************************************************
def f_x_fwd_numpy(X):
    '''
    The following function implement the test function using numpy.

    INPUTS:
    - X: List of input parameters. The lenght of X is the number of variables.
    '''
    return np.cos( np.sin( np.log( np.sum(X)**5 ) ) )
# =========================================================================================

# *****************************************************************************************
def f_x_fwd_oti(X):
    '''
    The following function implement the test function using OTI in full forward mode.

    INPUTS:
    - X: List of input parameters. The lenght of X is the number of variables.
    '''
    return oti.cos( oti.sin( oti.log( np.sum(X)**5 ) ) )
# =========================================================================================
# *****************************************************************************************
def f_x_rev_oti(X):
    '''
    The following function implement the test function using OTIs in 'reverse' mode.

    INPUTS:
    - X: List of input parameters. The lenght of X is the number of variables.
    '''
    w1 = np.sum(X)
    w2 = (w1.real + oti.e(1,order=w1.order))**5
    w3 = oti.log(w2.real + oti.e(1,order=w2.order))
    w4 = oti.sin(w3.real + oti.e(1,order=w3.order))
    w5 = oti.cos(w4.real + oti.e(1,order=w4.order))

    res = w5.rom_eval_object([1],[w4-w4.real])
    res = res.rom_eval_object([1],[w3-w3.real])
    res = res.rom_eval_object([1],[w2-w2.real])
    res = res.rom_eval_object([1],[w1-w1.real])
    return res
# =========================================================================================

In [3]:
order = 10
nvars = 10

X_r = [i+0.5 for i in range(1,nvars+1)]
X   = [i+0.5+oti.e(i,order=order) for i in range(1,nvars+1)]

print('Total number of derivatives to be computed: {}'.format(coti.ndir_total(nvars,order)))

Total number of derivatives to be computed: 184756


In [4]:
%timeit -r 4 f_x_fwd_numpy(X_r)

6.06 µs ± 422 ns per loop (mean ± std. dev. of 4 runs, 100,000 loops each)


In [5]:
%timeit -r 4 f_x_fwd_oti(X)

7.35 s ± 221 ms per loop (mean ± std. dev. of 4 runs, 1 loop each)


In [6]:
%timeit -r 4 f_x_rev_oti(X)

11.1 ms ± 563 µs per loop (mean ± std. dev. of 4 runs, 100 loops each)


In [7]:
# Ratios from the run:
convFact = {}
convFact['ns']=1e-9
convFact['µs']=1e-6
convFact['ms']=1e-3
convFact['s'] =1

# Change to the values obtained in your computer.

realRun   = 6.06*convFact['µs'] # 6.06 µs ± 422 ns per loop (mean ± std. dev. of 4 runs, 100,000 loops each)
otiFwdRun = 7.35*convFact['s']  # 7.35 s ± 221 ms  per loop (mean ± std. dev. of 4 runs, 1 loop each)
otiRevRun = 11.1*convFact['ms'] # 11.1 ms ± 563 µs per loop (mean ± std. dev. of 4 runs, 100 loops each)

print('fwd/real: {0:.2f}'.format(otiFwdRun/realRun))
print('rev/real: {0:.2f}'.format(otiRevRun/realRun))
print('fwd/rev : {0:.2f}x (speedup of reverse w.r.t fwd.)'.format(otiFwdRun/otiRevRun))

fwd/real: 1212871.29
rev/real: 1831.68
fwd/rev : 662.16x (speedup of reverse w.r.t fwd.)


In [8]:
f_fwd = f_x_fwd_oti(X)
f_rev = f_x_rev_oti(X)

In [9]:
diff = f_rev - f_fwd
print(diff)

 + 0 * e([1]) + 0 * e([2]) + 0 * e([3]) + 0 * e([4]) + ... 


In [10]:
diff.get_order_im(10)

 - 6.12156e-28 * e([[1,10]]) - 9.56731e-27 * e([[1,9],2]) - 7.73462e-26 * e([[1,8],[2,2]]) - 8.07794e-26 * e([[1,7],[2,3]]) + ... 

In [11]:
diff.get_order_im(5)

 - 2.64698e-23 * e([[1,5]]) - 9.52912e-22 * e([[1,4],2]) - 2.96462e-21 * e([[1,3],[2,2]]) - 3.38813e-21 * e([[1,2],[2,3]]) + ... 